# What is an API? (and what is not an API)


## 1. GET current weather (query parameters + API key)

```bash
curl --location 'https://api.openweathermap.org/data/2.5/weather?q=London&units=metric&appid=YOUR_API_KEY'
```

| Fragment                                           | Description                                                                                | API‑related? |
|----------------------------------------------------|--------------------------------------------------------------------------------------------|--------------|
| `curl`                                             | CLI tool to issue HTTP requests                                                            | ❌           |
| `--location`                                       | Client flag: follow HTTP `3xx` redirects                                                   | ❌           |
| `'https://api.openweathermap.org/data/2.5/weather` | **Endpoint URL**: scheme, host, path (includes version `2.5`)                               | ✅           |
| `?q=London&units=metric&appid=YOUR_API_KEY'`       | **Query parameters**: `q` (city), `units` (metric), `appid` (API key)                      | ✅           |
| *(implicit)*                                       | **Method**: `GET` (no `-X` or `--data` ⇒ default is GET)                                   | ✅           |
| *(none)*                                           | *(no headers or body needed for this public GET)*                                          | –            |

---

## 2. GET GitHub repos (pagination, versioning, auth)

```bash
curl \
  -H "Accept: application/vnd.github.v3+json" \
  -H "Authorization: Bearer ghp_ABC123…" \
  'https://api.github.com/users/octocat/repos?per_page=50&page=2'
```

| Fragment                                                             | Description                                                                | API‑related? |
|----------------------------------------------------------------------|----------------------------------------------------------------------------|--------------|
| `curl \`                                                             | CLI tool + line‑continuation                                               | ❌           |
| `-H "Accept: application/vnd.github.v3+json"`                        | **Header**: selects GitHub REST API v3 media type                           | ✅           |
| `-H "Authorization: Bearer ghp_ABC123…"`                              | **Header**: OAuth Bearer token for authentication                           | ✅           |
| `'https://api.github.com/users/octocat/repos`                        | **Endpoint**: templated path `/users/{username}/repos`                      | ✅           |
| `?per_page=50&page=2'`                                               | **Query params**: pagination controls                                        | ✅           |
| *(implicit GET)*                                                     | Method `GET`                                                                | ✅           |

---

## 3. POST create a new user (JSON body + versioned path)

```bash
curl -X POST https://api.example.com/v1/users \
     -H "Content-Type: application/json" \
     -H "Authorization: Bearer eyJ0eX…" \
     -d '{
           "name": "Alice",
           "email": "alice@example.com",
           "roles": ["admin","editor"]
         }'
```

| Fragment                                         | Description                                                      | API‑related? |
|--------------------------------------------------|------------------------------------------------------------------|--------------|
| `curl`                                           | CLI tool                                                         | ❌           |
| `-X POST`                                        | **Method**: `POST` creates a new resource                        | ✅           |
| `https://api.example.com/v1/users`               | **Endpoint**: version `v1` in the path                           | ✅           |
| `-H "Content-Type: application/json"`            | **Header**: declares JSON payload                                | ✅           |
| `-H "Authorization: Bearer eyJ0eX…"`              | **Header**: auth token                                           | ✅           |
| `-d '{ … }'`                                     | **Request body**: JSON object matching the user‑creation schema  | ✅           |

---

## 4. POST file upload (multipart/form-data)

```bash
curl -X POST https://files.example.com/v2/upload \
     -H "Authorization: ApiKey 12345" \
     -F 'metadata={"filename":"report.pdf","owner":"bob"};type=application/json' \
     -F 'file=@/path/to/report.pdf;type=application/pdf'
```

| Fragment                                                             | Description                                                        | API‑related? |
|----------------------------------------------------------------------|--------------------------------------------------------------------|--------------|
| `curl`                                                               | CLI tool                                                           | ❌           |
| `-X POST`                                                            | **Method**: `POST`                                                 | ✅           |
| `https://files.example.com/v2/upload`                                | **Endpoint**: `v2` in path                                         | ✅           |
| `-H "Authorization: ApiKey 12345"`                                   | **Header**: API key auth                                           | ✅           |
| `-F 'metadata=…;type=application/json'`                              | **Multipart field**: JSON metadata (must match API’s schema)       | ✅           |
| `-F 'file=@…report.pdf;type=application/pdf'`                        | **Multipart field**: the binary file                               | ✅           |

---

## 5. POST GraphQL query (single endpoint, payload‐driven)

```bash
curl -X POST https://api.myapp.com/graphql \
     -H "Content-Type: application/json" \
     -H "Authorization: Bearer abcdef" \
     -d '{
           "query": "query($id: ID!) { user(id: $id) { name email } }",
           "variables": { "id": "42" }
         }'
```

| Fragment                                         | Description                                                          | API‑related? |
|--------------------------------------------------|----------------------------------------------------------------------|--------------|
| `curl`                                           | CLI tool                                                             | ❌           |
| `-X POST`                                        | **Method**: `POST`                                                   | ✅           |
| `https://api.myapp.com/graphql`                  | **Endpoint**: single `/graphql` entrypoint                           | ✅           |
| `-H "Content-Type: application/json"`            | **Header**: JSON body                                                | ✅           |
| `-H "Authorization: Bearer abcdef"`              | **Header**: auth token                                               | ✅           |
| `-d '{ "query": …, "variables": … }'`            | **Payload**: GraphQL document + variables match the API schema       | ✅           |

---

### Summary of what “API” means vs. client tooling

- **API contract** items  
  - **Endpoint URL** (path, host, scheme, port)  
  - **HTTP method** (`GET`, `POST`, etc.)  
  - **Headers** (`Content-Type`, `Accept`, `Authorization`, custom media types)  
  - **Payload schema** (query params, JSON body, form fields, GraphQL documents)  

- **Not part of the API**  
  - The word `curl` and its flags like `--location`, `-F`, `-d` (they just tell `curl` how to talk HTTP)  
  - Shell line continuations (`\`), quoting style (`'…'` vs `"…"`)  
  - Local host/port for development (`127.0.0.1:5000`)—in production the same API contract often lives at `https://api…`  



# API Design Patterns: Python Libraries

## 1. NumPy: Creating and reshaping arrays

```python
import numpy as np

# Create a 2×3 array of floats
arr = np.array([[1, 2, 3], [4, 5, 6]], dtype=float)

# Reshape it to 3×2
reshaped = arr.reshape((3, 2))
```

| API piece                     | Details                                                                                 |
|--------------------------------|-----------------------------------------------------------------------------------------|
| **Function**                   | `np.array(object, dtype=None, order='K', ...)`                                          |
| **Parameters**                 | `object`: any sequence of numbers;<br>`dtype`: type of elements (here `float`).         |
| **Returns**                    | An `ndarray` of shape inferred from `object`.                                           |
| **Errors**                     | Raises `TypeError` if elements can’t be coerced to `dtype`;<br>`ValueError` if reshape size doesn’t match. |
| **Versioning**                 | Stable across NumPy 1.x series; major bump may change signature (rare).                 |

> **Non‑API bits**:  
> - The `import numpy as np` statement is just how you bring the library into your namespace.  
> - The variable names `arr`, `reshaped` and the Python assignment/use of them are not part of NumPy’s interface.

---

## 2. Pandas: Loading CSV and grouping data

```python
import pandas as pd

# Read a CSV of sales data
df = pd.read_csv("sales.csv", parse_dates=["date"])

# Compute total revenue per region
revenue = (
    df.groupby("region")["amount"]
      .sum()
      .reset_index(name="total_revenue")
)
```

| API piece                     | Details                                                                                 |
|--------------------------------|-----------------------------------------------------------------------------------------|
| **Function**                   | `pd.read_csv(filepath_or_buffer, **kwargs)`                                             |
| **Key kwargs**                 | `parse_dates`: list of column names to parse as dates;<br>`usecols`, `dtype`, etc.      |
| **Returns**                    | A `pandas.DataFrame`.                                                                   |
| **Method chaining**            |  
1. `df.groupby(by)` returns a `DataFrameGroupBy`.  
2. `["amount"]` selects a column.  
3. `.sum()` aggregates per group → `Series`.  
4. `.reset_index(name=…)` turns it back into a `DataFrame`. |
| **Errors**                     | `FileNotFoundError` if CSV missing;<br>`ParserError` if malformed CSV.                  |
| **Versioning**                 | Pandas 1.x maintains backward‑compat for `read_csv` and core groupby APIs.             |

> **Non‑API bits**:  
> - The string `"sales.csv"` and the file itself;  
> - The intermediate variable `df` and method chaining syntax are just Python code gluing calls together.

---

## 3. Requests: Calling a REST endpoint

```python
import requests

# Send JSON payload to login endpoint
resp = requests.post(
    "https://api.example.com/login",
    headers={"Content-Type": "application/json"},
    json={"username": "alice", "password": "s3cr3t"}
)

if resp.status_code == 200:
    token = resp.json()["token"]
else:
    raise RuntimeError(f"Login failed: {resp.status_code}")
```

| API piece                     | Details                                                                                           |
|--------------------------------|---------------------------------------------------------------------------------------------------|
| **Function**                   | `requests.post(url, **kwargs)`                                                                    |
| **Key kwargs**                 | `headers`: dict of HTTP headers;<br>`json`: Python dict → auto‑serialized.                       |
| **Returns**                    | `requests.Response` object with attributes: `status_code`, `.json()`, `.text`, `.headers`, etc. |
| **Errors**                     | Raises `requests.exceptions.RequestException` on network errors.                                 |
| **Versioning**                 | Requests 2.x maintains backward compatibility for `post` signature.                               |

> **Non‑API bits**:  
> - The URL `"https://api.example.com/login"` is specific to your service.  
> - The `if/else` logic and exception raising are user code around the API call.

---

## 4. Matplotlib: Plotting a graph

```python
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0, 10, 100)
y = np.sin(x)

# Plot with labels and legend
plt.plot(x, y, label="sin(x)")
plt.xlabel("x")
plt.ylabel("sin(x)")
plt.title("Sine Wave")
plt.legend()
plt.show()
```

| API piece                     | Details                                                                             |
|--------------------------------|-------------------------------------------------------------------------------------|
| **Functions**                  |  
- `plt.plot(x, y, **kwargs)`  
- `plt.xlabel(label)`  
- `plt.ylabel(label)`  
- `plt.title(label)`  
- `plt.legend(**kwargs)`  
- `plt.show()` |
| **Parameters**                 |  
- `x, y`: array‑like for plotting  
- `label`: legend text  
- Other kwargs: `color`, `linestyle`, `marker`, etc. |
| **Returns**                    | Most return a `Line2D` object or `Text` object; `show()` renders the figure.        |
| **Errors**                     | Raises `TypeError` if array sizes mismatch; `<FigureCanvas>` backends may raise on display. |

> **Non‑API bits**:  
> - Generating `x` and `y` via NumPy is separate (though you often combine APIs).  
> - The import alias `plt` is just a convention.

---

## 5. Scikit‑Learn: Training and evaluating a model

```python
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Split your data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the model
clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
```

| API piece                     | Details                                                                                                        |
|--------------------------------|----------------------------------------------------------------------------------------------------------------|
| **Function / Class**           |  
- `train_test_split(*arrays, test_size, random_state)`  
- `RandomForestClassifier(**hyperparams)`  
- `.fit(X, y)` trains the model  
- `.predict(X)` returns predictions  
- `classification_report(y_true, y_pred, **kwargs)` formats metrics |
| **Parameters**                 | Hyperparameters (`n_estimators`, `max_depth`), data arrays must be 2D for `X` and 1D for `y`.                 |
| **Returns**                    |  
- `train_test_split` → tuples of NumPy arrays  
- `fit` → the fitted estimator (in place)  
- `predict` → 1D NumPy array of labels  
- `classification_report` → formatted `str` (or dict if `output_dict=True`) |
| **Errors**                     | `ValueError` if shapes mismatch; `NotFittedError` if you call `predict` before `fit`.                           |
| **Versioning**                 | Maintains backward compatibility within major scikit‑learn releases.                                             |

> **Non‑API bits**:  
> - The variables `X`, `y`, and `random_state` choice;  
> - The `print()` call and how you handle the report string.

---

### Takeaways

- **Library APIs** (NumPy, pandas, requests, matplotlib, scikit‑learn) all expose **functions**, **classes**, **methods**, and **data types**—their “endpoints” are in‑process calls, not URLs.  
- Each call’s **signature** (name + parameters + return type + error conditions) is the API contract.  
- Everything else—the way you import, your variable names, control flow around the call—is gluing *to* the API, not part of it.

Armed with these examples, you can see that **any callable interface** (local or remote) conforms to the same principles: a named operation, clearly defined inputs, and guaranteed outputs or errors.